In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd
import numpy as np
import re
import string
from bs4 import BeautifulSoup
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding
from spacy.gold import GoldParse
from spacy.scorer import Scorer

In [0]:
cols = (4,6)
dataframe = pd.read_table("/content/task3_training.tsv",usecols=cols)
high_f = 0
adr_tag_train=[]
for i in dataframe.iterrows():
  tweet_txt=i[1][1].lower()
  tweet_txt=re.sub(r'[^\w\s]','',tweet_txt)
  tweet_txt=BeautifulSoup(tweet_txt).get_text()
  tweet_txt=tweet_txt.lower()
  tweet_txt=re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))','',tweet_txt)
  tweet_txt=re.sub('@[^\s]+','', tweet_txt)
  tweet_txt=re.sub(r'#([^\s]+)', r'\1', tweet_txt)
  tweet_txt=' '.join(re.sub("[\.\,\!\?\:\;\-\=]", " ", tweet_txt).split())
  tweet_txt=re.sub('[\s]+', ' ', tweet_txt)
  tweet_txt=tweet_txt.strip()
  txt=[]
  txt.append(tweet_txt)
  if not pd.isnull(i[1][0]):
    extra_txt=i[1][0].lower()
    extra_txt=re.sub(r'[^\w\s]','',extra_txt)
    start=tweet_txt.find(extra_txt)
    end=start+len(extra_txt)
    adr_data={"entities": [(start, end, "ADR")]}
  else:
    adr_data={"entities": []}
  tweet_txt.append(adr_data)
  adr_tag_train.append(tuple(txt))

In [0]:
cols = (4,6)
dataframe = pd.read_table("/content/task3_validation.tsv",usecols=cols)
val_data=[]
for i in dataframe.iterrows():
  tweet_txt=i[1][1].lower()
  tweet_txt=re.sub(r'[^\w\s]','',tweet_txt)
  tweet_txt=BeautifulSoup(tweet_txt).get_text()
  tweet_txt=tweet_txt.lower()
  tweet_txt=re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))','',tweet_txt)
  tweet_txt=re.sub('@[^\s]+','', tweet_txt)
  tweet_txt=re.sub(r'#([^\s]+)', r'\1', tweet_txt)
  tweet_txt=' '.join(re.sub("[\.\,\!\?\:\;\-\=]", " ", tweet_txt).split())
  tweet_txt=re.sub('[\s]+', ' ', tweet_txt)
  tweet_txt=tweet_txt.strip()
  txt=[]
  txt.append(tweet_txt)
  if not pd.isnull(i[1][0]):
    extra_txt=i[1][0].lower()
    extra_txt=re.sub(r'[^\w\s]','',extra_txt)
    start=tweet_txt.find(extra_txt)
    end=start+len(extra_txt)
    adr_data={"entities": [(start, end, "ADR")]}
  else:
    adr_data={"entities": []}
  txt.append(adr_data)
  val_data.append(tuple(txt))

In [8]:
adr_tag_train

[('seefischoral drugs for pyelonephritisciprofloxacin levofloxacin tmpsmz do not use nitrofurantoin for pyeloonly cystitisdavid_medinaf',
  {'entities': []}),
 ('happy for wellbutrin has similar effects as adderall trazodone is super promising for sleep but abilify can cause weight gain _',
  {'entities': []}),
 ('stilgarg im ok ty have an official diagnosis of bipolar now feeling ok at the moment lamotrigine has been increased having monotherapy',
  {'entities': []}),
 ('im soo depressed cymbalta couldnt help me', {'entities': []}),
 ('time for my daily afternoon relaxation ritual of smoking weed taking 2 mgs of clonazepam and 400 mg of seroquel xr',
  {'entities': []}),
 ('questionable xray result nothing a little levaquin wont help oh yeah and the cough meds with codeine',
  {'entities': []}),
 ('does nicotine gum actually help with quitting smoking', {'entities': []}),
 ('third night of diarrhea and first dose of norfloxacin gosh gonna die of dehydration',
  {'entities': []}),
 ('p

In [0]:
eval_res=0
def evaluate(ner_model,inp):
    scorer = Scorer()
    for input_,annot in inp:
        gold_text=ner_model.make_doc(input_)
        gold_p=GoldParse(gold_text, entities=annot.get('entities'))
        pred=ner_model(input_)
        scorer.score(pred,gold_p)
        global eval_res
        eval_res=scorer.scores
    return scorer.scores

In [0]:
def main(inp_tr_data,model=None,output_dir=None,n_iter=5):
    global high_f
    if model is not None:
        nlp = spacy.load(model)  
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  
        print("Initialized blank 'en' model")
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    else:
        ner = nlp.get_pipe("ner")
    ner.add_label("ADR")
    pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    with nlp.disable_pipes(*other_pipes):  
        optimizer = nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(inp_tr_data)
            losses = {}
            for data in inp_tr_data:
              if len(data) == 2:
                nlp.update([data[0]],[data[1]],drop=0.2,sgd=optimizer,losses=losses)
            batch_eval_result = evaluate(nlp,eval_data)
            print(batch_eval_result)
            if high_f < batch_eval_result['ents_per_type']['ADR']['f']:
              high_f = batch_eval_result['ents_per_type']['ADR']['f']          
              output_dir = Path(output_dir)
              if not output_dir.exists():
                  output_dir.mkdir()
              nlp.meta["name"] = "NER_Model"  
              nlp.to_disk(output_dir)
              print("Saved model to", output_dir)
            print("Losses", losses)
    for text, _ in inp_tr_data:
        doc = nlp(text)  

In [0]:
main(train_data=adr_tag_train,model='en_core_web_sm',output_dir="/content/",n_iter=50)

Loaded model 'en_core_web_sm'
{'uas': 0.0, 'las': 0.0, 'las_per_type': {'': {'p': 0.0, 'r': 0.0, 'f': 0.0}}, 'ents_p': 0.0, 'ents_r': 0.0, 'ents_f': 0.0, 'ents_per_type': {'ADR': {'p': 0.0, 'r': 0.0, 'f': 0.0}}, 'tags_acc': 0.0, 'token_acc': 100.0, 'textcat_score': 0.0, 'textcats_per_cat': {}}
Losses {'ner': 19856.291655256802}
{'uas': 0.0, 'las': 0.0, 'las_per_type': {'': {'p': 0.0, 'r': 0.0, 'f': 0.0}}, 'ents_p': 0.0, 'ents_r': 0.0, 'ents_f': 0.0, 'ents_per_type': {'ADR': {'p': 0.0, 'r': 0.0, 'f': 0.0}}, 'tags_acc': 0.0, 'token_acc': 100.0, 'textcat_score': 0.0, 'textcats_per_cat': {}}
Losses {'ner': 19512.17107790277}
{'uas': 0.0, 'las': 0.0, 'las_per_type': {'': {'p': 0.0, 'r': 0.0, 'f': 0.0}}, 'ents_p': 43.01075268817204, 'ents_r': 11.428571428571429, 'ents_f': 18.058690744920995, 'ents_per_type': {'ADR': {'p': 43.01075268817204, 'r': 11.428571428571429, 'f': 18.058690744920995}}, 'tags_acc': 0.0, 'token_acc': 100.0, 'textcat_score': 0.0, 'textcats_per_cat': {}}
Saved model to /co

In [0]:
eval_res

{'ents_f': 48.31625183016106,
 'ents_p': 49.549549549549546,
 'ents_per_type': {'ADR': {'f': 48.31625183016106,
   'p': 49.549549549549546,
   'r': 47.14285714285714}},
 'ents_r': 47.14285714285714,
 'las': 0.0,
 'las_per_type': {'': {'f': 0.0, 'p': 0.0, 'r': 0.0}},
 'tags_acc': 0.0,
 'textcat_score': 0.0,
 'textcats_per_cat': {},
 'token_acc': 100.0,
 'uas': 0.0}

In [11]:
f_score=eval_res['ents_per_type']['ADR']['f']
print(f_score)

48.31625183016106
